
<h1><center>Trends In Forest Recovery After Stand Replacing Disturbance: A Spectrotemporal Evaluation Of Productivity In Southeastern Pine Forests</center></h1>

<h4><center> Daniel J. Putnam </center></h4>

<center> For partial fulfillment of the reqiurements for the Master of Science degree </center>
<center> College of Natural Resources and Environment </center>
<center> Virginia Polytechnic Institute and State University </center>


## Analysis Preperation

### _Libraries_

In [2]:
import geemap
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from palettable.colorbrewer.diverging import RdYlGn_11 as NDVIpalette
from palettable.colorbrewer.sequential import YlOrRd_9 as LCMSpalette
ee.Initialize()

### _Imports_

In [3]:
LS5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") # landsat 5
LS7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") # landsat 7
LS8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") # landsat 8
LCMS = ee.ImageCollection("USFS/GTAC/LCMS/v2020-5") # Landcover Change Monitoring System
NLCD_col = ee.ImageCollection("USGS/NLCD_RELEASES/2016_REL") # national landcover Database
STATES = ee.FeatureCollection("TIGER/2018/States") # state polygon boundaries (probably don't need anymore)
ecoRegions = ee.FeatureCollection("EPA/Ecoregions/2013/L3") # EPA Ecoregions
loblolly = ee.FeatureCollection("users/dputnam21/USFS_loblollyRange") # USFS loblolly pine range within states of interest

### _Priliminary set-up_

In [4]:
# Expirementing with colorbrewer palettes
## palettes = ee.data.require('users/gena/packages:palettes') ## going to need to replace this section of code
#NDVIpalette = palettes.colorbrewer.RdYlGn[9]
#LCMSpalette = palettes.colorbrewer.YlOrRd[9]

# Creating sample date range for disturbances
startingD = ee.Date.fromYMD(1995,1,1)
endingD = ee.Date.fromYMD(2010,12,31)

### _Landcover/Landuse Mask_

In [5]:
# New NLCD/LCMS method
# retrieve NLCD for each year
NLCD_2001 = NLCD_col.filter(ee.Filter.eq('system:index', '2001')).first().select("landcover")
NLCD_2004 = NLCD_col.filter(ee.Filter.eq('system:index', '2004')).first().select("landcover")
NLCD_2006 = NLCD_col.filter(ee.Filter.eq('system:index', '2006')).first().select("landcover")
NLCD_2008 = NLCD_col.filter(ee.Filter.eq('system:index', '2008')).first().select("landcover")
NLCD_2011 = NLCD_col.filter(ee.Filter.eq('system:index', '2011')).first().select("landcover")
NLCD_2013 = NLCD_col.filter(ee.Filter.eq('system:index', '2013')).first().select("landcover")
NLCD_2016 = NLCD_col.filter(ee.Filter.eq('system:index', '2016')).first().select("landcover")

# retrieve LCMS landuse classification
LCMSlanduseCol = LCMS.select("Land_Use")

# combine to image collection
NLCDlandcover_col = ee.ImageCollection(ee.List([NLCD_2001,NLCD_2004,NLCD_2006,NLCD_2008,NLCD_2011,NLCD_2013,NLCD_2016]))

# finding the most frequently identified landcover class
NLCDmode = NLCDlandcover_col.reduce(ee.Reducer.mode())
LCMSmode = LCMSlanduseCol.reduce(ee.Reducer.mode())

# combining the two layers into a mask
landCoverMask = LCMSmode.select('Land_Use_mode').eq(ee.Image(3))

# Pixels must have forest landuse classification from LCMS, and evergreen (NLCD), or woody wetland (NLCD)
landCoverMask = landCoverMask.updateMask(landCoverMask.And(NLCDmode.eq(ee.Image(42)) \
                                                           .Or(NLCDmode.eq(ee.Image(90))) \
                                                            ))

landCoverMask = landCoverMask.clip(loblolly) # clip mask to study boundaries for better loading

### _Landsat Preprocessing_

In [6]:
# Cloud masking based on the QA band : code taken from landsat example in data catalog in EE
def LScloudMask(image):
  qa = image.select('pixel_qa')
    # removing cloud pixels if confiance is high, cloud shadow, snow
  cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)) \
            .Or(qa.bitwiseAnd(1 << 3)) \
            .Or(qa.bitwiseAnd(1 << 4))
  return image.updateMask(cloud.Not())

# Going to try removing the coverage overlap between LS5 and LS8 to try and fix some issues
#LS5 = LS5.filterDate(start = '1984-01-01',opt_end = ee.Date('2013-04-11'))
#LS8 = LS8.filterDate(start = ee.Date('2013-04-11'))

# Lansat 5/7 & 8 differ in their band labeling, need to select the bands I'm going to use and rename them to
# match each other before merging collections : bands I need [red,green,NIR,SWIR1,SWIR2]    
LS8BandNames = ee.List(['B4','B3','B5','B6','B7','pixel_qa'])
NewBandNames = ee.List(['B3','B2','B4','B5','B7','pixel_qa'])
LS8 = LS8.select(LS8BandNames,NewBandNames)

# Adding a function to calculate and add an NDVI band for a single image
def addNDVI(image):
  ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
  return image.addBands(ndvi)

# Adding a function to calculate and add an NBR band for a single image.
def addNBR(image):
  nbr = image.normalizedDifference(['B4', 'B7']).rename('NBR')
  return image.addBands(nbr)

# Adding a function to calculate and add an MBI band for a single image.
def addMBI(image):
  MBI = image.expression(
  "MBI = ((b('B5') - b('B7') - b('B4')) / (b('B5') + b('B7') + b('B4'))) + 0.5")
  return image.addBands(MBI)

# adding the cloud mask per generation
LS5 = LS5.map(LScloudMask)
LS7 = LS7.map(LScloudMask)
LS8 = LS8.map(LScloudMask)

# merging the landsat 5 and 7 collections
LS_stack = LS5.merge(LS8)
LS_stack = LS_stack.merge(LS7)

# data reduction on the image stack
LS_stack = LS_stack.filterBounds(loblolly)

# Adding the indices to the filtered combined Landsat collection
LS_stack_wVI = LS_stack.map(addNDVI)
LS_stack_wVI = LS_stack_wVI.map(addNBR)
LS_stack_wVI = LS_stack_wVI.map(addMBI)

---

## Stand Selection Methods

### _LCMS Fast change method_

In [7]:
# Using the LCMS Change metric to identify harvest areas in contrast to the max VI method
# Filtering LCMS for the region and timeframe
LCMSchange = LCMS.select('Change')
LCMSchange = LCMSchange.filterDate(startingD,endingD)

# a function to apply the landcover mask to the LCMS image stack and select only fast change pixels
# also remaps values representing fast change to '1' and adds a band indicating year of disturbance
def NLCDmask(image):
    fastChange = image.updateMask(image.eq(3))
    fastChange = fastChange.remap([3],[1],bandName = 'Change')
    fastChangeMasked = fastChange.updateMask(landCoverMask)
    return fastChangeMasked

# applying the function to the LCMS
FC_stack = LCMSchange.map(NLCDmask)

### _Connected Pixel (Min stand size) mask_

In [8]:
# function to apply a connected pixel mask to the input image
def conectPixls(InImage,minArea,maxPixels):
    pixelCount = InImage.connectedPixelCount(maxPixels,False)
    minPixelCount = ee.Image(minArea).divide(ee.Image.pixelArea())
    outImage = InImage.updateMask(pixelCount.gte(minPixelCount))
    return outImage

# a function to be mapped accross an image collection and annually apply the connected pixels mask, also creates an
# additional band to store the year of disturbance for each pixel
def annualConectPixls(image):
    conectPixlsMasked = conectPixls(image,37800,250)
    imgYear = image.date().get('year')
    imgYearBand = ee.Image.constant(imgYear).uint16().rename('ChangeY')
    imgYearBand = imgYearBand.updateMask(conectPixlsMasked)
    return conectPixlsMasked.addBands(imgYearBand)

FC_final = FC_stack.map(annualConectPixls)

In [9]:
# creating the summary images
FC_final_changeN = FC_final.select('remapped').reduce(ee.Reducer.sum()) #.reproject(FC_stack.select('remapped').first().projection())
FC_final_1stYear = FC_final.select('ChangeY').reduce(ee.Reducer.min()) #.reproject(FC_stack.select('remapped').first().projection())

---

## New Automatic Stand Selection Method

### _Creating Sampling Areas Using Ecoregions_

In [10]:
# Limit ecoregions by overlap with loblolly range
loblollyEcoRegions = ecoRegions.filterBounds(loblolly)

# Function to convert the ecoregion code to an integer value
def convertPropertyToBand(feat):
    feat = ee.Feature(feat)
    prop = feat.get('us_l3code')
    propInt = ee.Number.parse(prop).toInt()
    feat = feat.set({'numericL3ecocode':propInt})
    return feat
loblollyEcoRegions = loblollyEcoRegions.map(convertPropertyToBand)

# Need to convert ecoregion feature collection and the property to integer in order for it to be used 
#     as the 'classBand' in the stratifiedSample fucntion
ecoregionImage = ee.Image(loblollyEcoRegions.reduceToImage(['numericL3ecocode'],ee.Reducer.first()))
ecoregionImage = ecoregionImage.cast({'first':'int8'})
ecoregionImage = ecoregionImage.clipToCollection(loblolly)

# An image representing pixels that are fast change and meet the landcover reqiurements
potentialSamples = FC_final_changeN.updateMask(FC_final_changeN.lte(2))

# Adding ecoregion code as band to potential sample pixels
potentialSamples = potentialSamples.addBands(ecoregionImage.select('first').rename('numericL3ecocode'))

### _Creating Random Sample Points_

In [ ]:
# Going to try just using the export table function to drive
samplePoints = potentialSamples.stratifiedSample(numPoints = 50,
                                                 region = loblolly,
                                                 classBand = 'numericL3ecocode',
                                                 scale = 30,
                                                 seed = 5,
                                                 dropNulls = True,
                                                 geometries = True,
                                                 )

# The export process will take about 15 minutes to complete
geemap.ee_export_vector_to_drive(samplePoints, 'stratifiedSamplePoints', 'EarthEngine_Exports', file_format='shp', selectors=None)

In [13]:
# Importing the points created in the above cell
samplePoints = ee.FeatureCollection('users/dputnam21/stratifiedSamplePoints_03022022')

---

### Displaying images on the map

In [15]:
# LCMS landcover palette
LCMSlcPalette = ['efff6b','ff2ff8','1b9d0c','97ffff','a1a1a1','c2b34a','1B1716']

Map = geemap.Map()
Map.centerObject(loblolly,7)

# This is the bottom of the layer order

Map.addLayer(ecoregionImage.select('first'), vis_params = {'palette': LCMSlcPalette, 'min': 45, 'max':75}, name = 'Ecoregion Code Image')
Map.addLayer(loblolly, vis_params = {'color' :'red'}, name = 'Study Area', shown = False)
Map.addLayer(landCoverMask, vis_params = {'palette': ['white','2ca25f'], 'min' : 0, 'max' : 1}, name = 'new landcover mask', shown = True)
Map.addLayer(FC_final_1stYear,{'palette':['edf8b1','7fcdbb','2c7fb8'],'min' : 1995, 'max' : 2010},'LCMS Fast Change Year',False)
Map.addLayer(FC_final_changeN,{'palette':['fee0d2','fc9272','de2d26'],'min':1,'max':3},'LCMS Fast Change Count',False)
Map.addLayer(potentialSamples.select('remapped_sum'),{'palette':['fee0d2','fc9272','de2d26'],'min':1,'max':5}, name = 'Potential Sample Pixels')
Map.addLayer(samplePoints,{'color':'red'}, name = 'Stratified Random Samples')



# This is the top of the layer order
Map.addLayerControl()
Map

Map(center=[33.55547594326177, -83.81053005836175], controls=(WidgetControl(options=['position', 'transparent_…

### Export of stand attributes

In [16]:
# adding unique ID to each point

# Adding environmental variables as attributes for each stand


### New compositing method

In [ ]:
# enter analysis parameters
compositeMonthN = 11
outputIndex = 'NBR'
compositeStat = 'median'

# prep for function
chart_VI = LS_stack_wVI.filter(ee.Filter.calendarRange(compositeMonthN,compositeMonthN,'month'));

years = ee.List.sequence(1984, 2020)

In [ ]:
from datetime import datetime

start = datetime.now() # figuring out how long this takes to run

def perFeatureValueExtraction(index,comMonth,vegIndex,comStat) :
    
    aFeature = ee.Feature(stands2.filter(ee.Filter.eq('UniqueID', index)).first())
    inFeature = aFeature.geometry()

    local_chart_VI = chart_VI.filterBounds(inFeature)

    # A function to be mapped over a feature colllection, extracts 37 index values for each feature, returns them as a
    def seasonalReduction_extraction (y):
        filteredColl = local_chart_VI.filter(ee.Filter.calendarRange(y, y, 'year'))
        singleImage = filteredColl.select(['NDVI','NBR','MBI']).reduce(ee.Reducer.median()) ## CHANGE COMPOSITE STAT HERE ###
        outputImage = singleImage.set('system:time_start', ee.Date.fromYMD(y,comMonth, 1).millis())
        valDict = outputImage.reduceRegion(reducer = ee.Reducer.mean(), geometry = inFeature, scale = 30)
        NBRval = ee.Number(valDict.get(ee.String(vegIndex+'_'+compositeStat),defaultValue = 0.0))
        return ee.List([NBRval, 0.0]).reduce(ee.Reducer.firstNonNull())

    outputList = ee.List(years.map(seasonalReduction_extraction,False)).getInfo()
    return outputList

# calling the function
NumPlots = stands2.size().getInfo()
ListOfValLists = []

for index in range(NumPlots) :
    #aFeature = ee.Feature(stands2.filter(ee.Filter.eq('system:index', 0)).first())
    aList = perFeatureValueExtraction(index,compositeMonthN,outputIndex,compositeStat)
    ListOfValLists.append(aList)
    
end = datetime.now() # effectively ending the timer
duration = end - start

# nicely printing the ellapsed time
print('this function took this long to run :',duration,'\n')
timeList = str(duration).split(':')
print('The time elapsed during this execution of this operation was :','\n',
     timeList[0],'Hour(s)','\n',
     timeList[1],'Minute(s)','\n',
      'and',round(float(timeList[2]),ndigits = 0),'Seconds'
     )

# nicely displaying the raw data
standNum = 0
for alist in ListOfValLists :
    print('Stand',standNum,'\n',alist,'\n',('-'*124))
    standNum += 1

In [ ]:
# creating arrays for rows and columns of the table
stand_nums = range(0,stands2.size().getInfo())
imageYears = years.getInfo()

# client side replacement of null value (0.0) with null object
NDVIvals = ListOfValLists
for i in range(0,len(NDVIvals)):
    for i2 in range(0,imageYears) :
        if NDVIvals[i][i2] == 0 :
            NDVIvals[i][i2] = None

In [ ]:
# Creating the dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

NDVItsDF = pd.DataFrame(index = stand_nums, columns = imageYears)
valIndex = 0
for r in range(0,len(stand_nums)):
    for c in range(0,len(imageYears)):
        NDVItsDF.iloc[r,c] = NDVIvals[r][c]
        valIndex += 1
        
for col in NDVItsDF:
    NDVItsDF[col] = pd.to_numeric(NDVItsDF[col], errors='coerce')
NDVItsDF

### Creating time series plots

In [ ]:
%matplotlib inline
import scipy as sp
import scipy.signal as scisig

# interpolation of null values
plotDF = NDVItsDF.interpolate(axis = 'columns',method = 'linear')

# setting up for time-series plots
fig, axs = plt.subplots(7, 3, sharex=False, sharey=True, figsize = (16,25))

for i, ax in enumerate(fig.axes):
    ax.plot(plotDF.iloc[i,].transpose(),label = "Interpolated")
    ax.plot(NDVItsDF.iloc[i,].transpose(),label = "Original")
    ax.scatter(imageYears,NDVItsDF.iloc[i,], color = 'orange',s = 20)
    ax.legend(loc='lower right')
    ax.set_title("Stand"+' '+str(i)+' '+"Time-Series")
    
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, wspace=0.3, hspace=0.5)
fig.text(0.5, 0.04, 'Year', ha='center', va='center')
fig.text(0.06, 0.5, (outputIndex+' '+'value'), ha='center', va='center', rotation='vertical')

In [ ]:
plotDF

plotDF.to_csv(path_or_buf="C:/R_workspace/timeSeriesDF.csv", sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')